In [1]:
#!pip install --upgrade git+https://github.com/EmGarr/kerod.git

In [2]:
#%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found')
else:
    print('Found GPU at: {}'.format(device_name))

GPU device not found


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import  functools
import  tensorflow as tf
import  tensorflow_datasets as tfds
from    tensorflow.keras.utils import to_categorical
import  matplotlib.pyplot as plt
from    tensorflow.keras import layers
from    tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D




In [5]:
data_dir='/home/suphale/coco'
N_LABELS = 80
N_EPOCHS = 1
TRAIN_MODE = False
split_train = "train[:1%]"
split_val = "validation[:1%]"
h_image_height = 299
h_image_width = 299


In [6]:
class BoxField:
    BOXES = 'bbox'
    KEYPOINTS = 'keypoints'
    LABELS = 'label'
    MASKS = 'masks'
    NUM_BOXES = 'num_boxes'
    SCORES = 'scores'
    WEIGHTS = 'weights'

class DatasetField:
    IMAGES = 'images'
    IMAGES_INFO = 'images_information'
    IMAGES_PMASK = 'images_padding_mask'

def my_preprocess(inputs):
    image = inputs['image']
    image = tf.image.resize(image, (h_image_height, h_image_width))
    image = tf.cast(image, tf.float32)
    image /= 127.5
    image -= 1.

    targets = inputs['objects']

    image_information = tf.cast(tf.shape(image)[:2], dtype=tf.float32)

    inputs = {DatasetField.IMAGES: image, DatasetField.IMAGES_INFO: image_information}

    # ground_truths = {
    #     BoxField.BOXES: targets[BoxField.BOXES] * tf.tile(image_information[tf.newaxis], [1, 2]),
    #     BoxField.LABELS: tf.cast(targets[BoxField.LABELS], tf.int32),
    #     BoxField.NUM_BOXES: tf.shape(targets[BoxField.LABELS]),
    #     BoxField.WEIGHTS: tf.fill(tf.shape(targets[BoxField.LABELS]), 1.0)
    # }
    ground_truths = tf.cast(targets[BoxField.LABELS], tf.int32)
    ground_truths = tf.one_hot(ground_truths, depth=N_LABELS, dtype=tf.int32)
    ground_truths = tf.reduce_sum(ground_truths, 0)
    ground_truths = tf.greater( ground_truths, tf.constant( 0 ) )    
    ground_truths = tf.where (ground_truths, 1, 0) 
    return image, ground_truths

def expand_dims_for_single_batch(image, ground_truths):
    image = tf.expand_dims(image, axis=0)
    ground_truths = tf.expand_dims(ground_truths, axis=0)
    return image, ground_truths


In [7]:
ds_train, ds_info = tfds.load(name="coco/2017", split=split_train, data_dir=data_dir, shuffle_files=True, download=False, with_info=True)
ds_train = ds_train.map(functools.partial(my_preprocess), num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.map(expand_dims_for_single_batch, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_val = tfds.load(name="coco/2017", split=split_val, data_dir=data_dir, shuffle_files=True, download=False)
ds_val = ds_val.map(functools.partial(my_preprocess), num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_val = ds_val.map(expand_dims_for_single_batch, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_val = ds_val.prefetch(tf.data.experimental.AUTOTUNE)


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


# Load and train the network


In [8]:
# Find total number of classes in the coco dataset
classes = ds_info.features['objects']['label'].names
num_classes = len(classes)
print(num_classes)

80


In [9]:
image_model = tf.keras.applications.InceptionV3(include_top=False,weights='imagenet')

x = image_model.output
x = GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
predictions = tf.keras.layers.Dense(80, activation='sigmoid')(x)

model = tf.keras.Model(inputs=image_model.input, outputs=predictions)

for layer in image_model.layers:
    layer.trainable = False

# model.compile(optimizer='rmsprop', loss=ncce, metrics=['accuracy'])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [10]:
# for layer in model.layers:
#     print(layer.trainable)

In [11]:
if (TRAIN_MODE == True):
    callbacks = [
        TensorBoard(),
        ModelCheckpoint('checkpoints/')
    ]

    model.fit(ds_train, validation_data=ds_val, epochs=N_EPOCHS, callbacks=callbacks)
    # Save the weights for the serving
    model.save_weights('/home/suphale/WorkSpace/temp_model/coco_classification_weights.h5')
else:
    model.load_weights('/home/suphale/WorkSpace/saved_model/coco_classification_weights.h5')    


## Visualisation of few images

In [12]:
for test_index in range(10):
    sample_img_batch, sample_cap_batch = next(iter(ds_val))
    s = model(sample_img_batch)
    # plt.imshow(tf.squeeze(sample_img_batch, [0]))

    print("Reference:")
    n = sample_cap_batch.numpy()
    index = 0
    for x in n[0]:
        if x > 0.1:
            print("%s," % (classes[index]), end=' ')
        index += 1
    print("")
    print("Prediction:")
    n = s.numpy()
    index = 0
    for x in n[0]:
        if x > 0.5:
            print("%s(%.2f)," % (classes[index],x), end=' ')
        index += 1
    print("")


Reference:
person, tennis racket, 
Prediction:
person(0.99), sports ball(0.52), tennis racket(1.00), 
Reference:
remote, book, 
Prediction:
bottle(0.54), 
Reference:
person, car, handbag, 
Prediction:
person(0.96), skateboard(0.89), 
Reference:
person, car, handbag, 
Prediction:
person(0.96), skateboard(0.89), 
Reference:
person, tennis racket, 
Prediction:
person(0.99), sports ball(0.52), tennis racket(1.00), 
Reference:
remote, book, 
Prediction:
bottle(0.54), 
Reference:
person, tennis racket, 
Prediction:
person(0.99), sports ball(0.52), tennis racket(1.00), 
Reference:
remote, book, 
Prediction:
bottle(0.54), 
Reference:
person, tennis racket, 
Prediction:
person(0.99), sports ball(0.52), tennis racket(1.00), 
Reference:
remote, book, 
Prediction:
bottle(0.54), 


## Tensorboard

In [13]:
TensorBoard = False
if(TensorBoard == True):
    # Load TENSORBOARD
    %load_ext tensorboard
    # Start TENSORBOARD
    %tensorboard --logdir logs

In [14]:
from tensorflow import keras  # or import keras for standalone version
from tensorflow.keras.layers import Input

org_model = tf.keras.models.load_model('model')


In [15]:
model_config = org_model.get_config()

In [16]:
max_layer_index = len(model_config['layers']) - 1

In [17]:
split_layer = 4

new_head_model_config = {}
new_head_model_config['name'] = 'head_model'
new_head_model_config['layers'] = []
new_head_model_config['input_layers'] = [[model_config['layers'][0]['name'],0,0]]
new_head_model_config['output_layers'] = [[model_config['layers'][split_layer-1]['name'],0,0]]

for index in range(split_layer):
    print("%d %s" % (index, model_config['layers'][index]['name']) )
    new_head_model_config['layers'].append(model_config['layers'][index])

0 input_1
1 conv2d
2 batch_normalization
3 activation


In [18]:
import copy

new_tail_model_config = copy.deepcopy(model_config)
new_tail_model_config['name'] = 'tail_model'
new_tail_model_config['input_layers'] = [[model_config['layers'][split_layer]['name'],0,0]]
new_tail_model_config['output_layers'] = [[model_config['layers'][max_layer_index]['name'],0,0]]

In [19]:
# tuple(org_model.layers[0].output.shape)

In [20]:
new_input_layer = {
                      'name': 'new_input',
                      'class_name': 'InputLayer',
                      'config': {
                          'batch_input_shape': tuple(org_model.layers[split_layer].output.shape),
                          'dtype': 'float32',
                          'sparse': False,
                          'name': 'new_input'
                      },
                      'inbound_nodes': []
                  }
new_tail_model_config['layers'][0] = new_input_layer

In [21]:
for index in range(1,split_layer):
    print("%d %s" % (index, new_tail_model_config['layers'][1]['name']) )
    # new_tail_model_config['layers'].append(model_config['layers'][index])
    new_tail_model_config['layers'].pop(1)

1 conv2d
2 batch_normalization
3 activation


In [22]:
# input_layer_name = model_config['layers'][0]['name']
new_tail_model_config['layers'][1]['inbound_nodes'] = [[['new_input', 0, 0, {}]]]
new_tail_model_config['input_layers'] = [['new_input', 0, 0]]

In [23]:
import pprint
with open('model_config.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    # fh.write(str(model_config))
    pp = pprint.PrettyPrinter(indent=4, stream=fh)
    pp.pprint(str(model_config))
with open('new_head_model_config.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    # fh.write(str(new_head_model_config))
    pp = pprint.PrettyPrinter(indent=4, stream=fh)
    pp.pprint(str(new_head_model_config))
with open('new_tail_model_config.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    # fh.write(str(new_head_model_config))
    pp = pprint.PrettyPrinter(indent=4, stream=fh)
    pp.pprint(str(new_tail_model_config))


In [24]:
new_head_model = org_model.__class__.from_config(new_head_model_config, custom_objects={})

In [25]:
new_tail_model = org_model.__class__.from_config(new_tail_model_config, custom_objects={})

In [26]:
with open('org_model.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    org_model.summary(print_fn=lambda x: fh.write(x + '\n'), line_length=150)

with open('new_head_model.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    new_head_model.summary(print_fn=lambda x: fh.write(x + '\n'), line_length=150)

with open('new_tail_model.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    new_tail_model.summary(print_fn=lambda x: fh.write(x + '\n'), line_length=150)


In [27]:
for index, layer in enumerate(org_model.layers[:split_layer]):
    # print("[%d] %s %s" % (index, layer.name, str(np.shape(weight))))
    weight = layer.get_weights()
    new_head_model_layer = new_head_model.layers[index]
    new_head_model_layer.set_weights(weight)


In [28]:
for index, layer in enumerate(org_model.layers[split_layer:max_layer_index+1]):
    weight = layer.get_weights()
    # print("[%d] %s %s" % (index, layer.name, str(np.shape(weight))))
    new_tail_model_layer = new_tail_model.layers[index+1]
    # print("[%d] %s %s" % (index, new_tail_model_layer.name, str(np.shape(weight))))
    new_tail_model_layer.set_weights(weight)

In [29]:
hidden_layer = new_head_model(sample_img_batch)
print(tf.shape(hidden_layer))

# new_head_model.summary(line_length=150)
predictions = new_tail_model(hidden_layer)
# # head_model.summary(line_length=150)
# print(tf.shape(predictions))

tf.Tensor([  1 149 149  32], shape=(4,), dtype=int32)


In [30]:
for test_index in range(10):
    sample_img_batch, sample_cap_batch = next(iter(ds_val))
    h = new_head_model(sample_img_batch)
    s = new_tail_model(h)

    # plt.imshow(tf.squeeze(sample_img_batch, [0]))

    print("---")
    print("Reference  :", end=' ')
    n = sample_cap_batch.numpy()
    index = 0
    for x in n[0]:
        if x > 0.1:
            print("%s," % (classes[index]), end=' ')
        index += 1
    print("")
    print("Prediction :", end=' ')
    n = s.numpy()
    index = 0
    for x in n[0]:
        if x > 0.5:
            print("%s(%.2f)," % (classes[index],x), end=' ')
        index += 1
    print("")

---
Reference  : person, bus, book, 
Prediction : bus(0.99), 
---
Reference  : person, bus, book, 
Prediction : bus(0.99), 
---
Reference  : person, bus, book, 
Prediction : bus(0.99), 
---
Reference  : person, bus, book, 
Prediction : bus(0.99), 
---
Reference  : person, bus, book, 
Prediction : bus(0.99), 
---
Reference  : person, car, traffic light, stop sign, 
Prediction : car(0.92), traffic light(0.67), 
---
Reference  : bottle, cup, oven, sink, 
Prediction : oven(0.53), sink(0.63), 
---
Reference  : remote, book, 
Prediction : bottle(0.54), 
---
Reference  : person, car, handbag, 
Prediction : person(0.96), skateboard(0.89), 
---
Reference  : person, tennis racket, 
Prediction : person(0.99), sports ball(0.52), tennis racket(1.00), 
